In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("canerkonuk/youtube-trending-videos-global")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/canerkonuk/youtube-trending-videos-global/versions/143


In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix


In [3]:
# List files in the downloaded directory
files = os.listdir(path)
print("Files in dataset:", files)

# Assuming the dataset contains a CSV file, read it into a Pandas DataFrame
csv_file = [file for file in files if file.endswith(".csv")][0]  # Pick the first CSV file
df = pd.read_csv(os.path.join(path, csv_file))

# Display the first few rows
df.head()


Files in dataset: ['youtube_trending_videos_global.csv']


<ipython-input-3-4bbfc8a50ebd>:7: DtypeWarning: Columns (13,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(path, csv_file))


,video_id,video_published_at,video_trending__date,video_trending_country,channel_id,video_title,video_description,video_default_thumbnail,video_category_id,video_tags,...,channel_description,channel_custom_url,channel_published_at,channel_country,channel_view_count,channel_subscriber_count,channel_have_hidden_subscribers,channel_video_count,channel_localized_title,channel_localized_description
0,bB3-CUMERIU,2024-10-11T00:00:06Z,2024.10.12,United Arab Emirates,UCNYi_zGmR519r5gYdOKLTjQ,JENNIE - Mantra (Official Music Video),Official music video for “Mantra” by JENNIE ou...,https://i.ytimg.com/vi/bB3-CUMERIU/default.jpg,Music,"JENNIE,Mantra,제니,Jennie Mantra,Jennie from Bla...",...,NaN,@jennierubyjane,2021-01-13T06:19:55.86689Z,NaN,464615150.0,11600000.0,False,43.0,JENNIE,NaN
1,5ObJt_71AYc,2024-10-11T02:59:21Z,2024.10.12,United Arab Emirates,UCzU8-lZlRfkV3nj0RzAZdrQ,CHILE vs. BRASIL [1-2] | RESUMEN | ELIMINATORI...,#Chile #Brasil #eliminatoriassudamericanas #Cr...,https://i.ytimg.com/vi/5ObJt_71AYc/default.jpg,Sports,"fútbol,pelota,gol,copa,libertadores,sudamerica...",...,Canal Oficial de la Confederación Sudamericana...,@conmebol,2014-02-19T20:24:31Z,NaN,399046746.0,1610000.0,False,4637.0,CONMEBOL,Canal Oficial de la Confederación Sudamericana...
2,zfb0whgBBA8,2024-10-11T11:07:25Z,2024.10.12,United Arab Emirates,UCgGYPnVJytkr6sVNLQ-l0zQ,تحدي الحقيبة مع ابو عبير 🔥 | سلسلتنا #2,"1,375,547 مشترك❤️\n\nـــــــــــــــــــــــــ...",https://i.ytimg.com/vi/zfb0whgBBA8/default.jpg,Gaming,"تشكيله,فيفا,فيفا20,التمت,درافت,زلاتان,ام,جي,مل...",...,صانع محتوى وانزل مقاطع بشكل غير منظم 🌚,@mrlle99,2012-08-01T16:24:26Z,Saudi Arabia,114331110.0,1380000.0,False,314.0,LLE عبدالاله,صانع محتوى وانزل مقاطع بشكل غير منظم 🌚
3,SJfoPdeOPCQ,2024-10-11T00:10:10Z,2024.10.12,United Arab Emirates,UCzU8-lZlRfkV3nj0RzAZdrQ,VENEZUELA vs. ARGENTINA [1-1] | RESUMEN | ELIM...,#Venezuela #Argentina #eliminatoriassudamerica...,https://i.ytimg.com/vi/SJfoPdeOPCQ/default.jpg,Sports,"fútbol,pelota,gol,copa,libertadores,sudamerica...",...,Canal Oficial de la Confederación Sudamericana...,@conmebol,2014-02-19T20:24:31Z,NaN,399046746.0,1610000.0,False,4637.0,CONMEBOL,Canal Oficial de la Confederación Sudamericana...
4,UVb6QOKy0bI,2024-10-09T12:30:27Z,2024.10.12,United Arab Emirates,UCOzubmwpVZI7gD0Jf7Bk3Aw,Bougainvillea Trailer | October 17 | Jyothirma...,Presenting The Official Trailer of Bougainvill...,https://i.ytimg.com/vi/UVb6QOKy0bI/default.jpg,Film & Animation,NaN,...,Official YouTube Channel Of Amal Neerad Produc...,@amalneeradproductions5264,2017-05-31T14:31:01Z,NaN,19991522.0,40600.0,False,56.0,Amal Neerad Productions,Official YouTube Channel Of Amal Neerad Produc...


In [4]:
df.isnull().sum()

,0
video_id,5
video_published_at,0
video_trending__date,0
video_trending_country,0
channel_id,0
video_title,0
video_description,317392
video_default_thumbnail,0
video_category_id,1438
video_tags,643984


In [5]:
df['video_category_id'].mode()

,video_category_id
0,Entertainment


In [6]:
df["video_category_id"].fillna(df['video_category_id'].mode(),inplace=True)

<ipython-input-6-bf15ccd4401a>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["video_category_id"].fillna(df['video_category_id'].mode(),inplace=True)


In [7]:
df["video_category_id"].fillna(df['video_category_id'].mode()[0], inplace=True)

In [8]:
df['video_category_id'].isnull().sum()

0

In [9]:
df.isnull().sum()

,0
video_id,5
video_published_at,0
video_trending__date,0
video_trending_country,0
channel_id,0
video_title,0
video_description,317392
video_default_thumbnail,0
video_category_id,0
video_tags,643984


In [10]:
df["channel_view_count"].fillna(df["channel_view_count"].median(), inplace=True)
df["channel_subscriber_count"].fillna(df["channel_subscriber_count"].median(), inplace=True)


<ipython-input-10-371f64e4bcd9>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["channel_view_count"].fillna(df["channel_view_count"].median(), inplace=True)
<ipython-input-10-371f64e4bcd9>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

In [11]:
df["video_view_count"].fillna(df["video_view_count"].median(), inplace=True)
df["video_like_count"].fillna(df["video_view_count"].median(), inplace=True)

<ipython-input-11-dd41d6438ae8>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["video_view_count"].fillna(df["video_view_count"].median(), inplace=True)
<ipython-input-11-dd41d6438ae8>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(val

In [12]:
df.isnull().sum()

,0
video_id,5
video_published_at,0
video_trending__date,0
video_trending_country,0
channel_id,0
video_title,0
video_description,317392
video_default_thumbnail,0
video_category_id,0
video_tags,643984


In [13]:
# Select every other column
df.iloc[:, ::2]  # Selects columns at even indices (0, 2, 4, 6, 8)



,video_id,video_trending__date,channel_id,video_description,video_category_id,video_duration,video_definition,video_view_count,video_comment_count,channel_description,channel_published_at,channel_view_count,channel_have_hidden_subscribers,channel_localized_title
0,bB3-CUMERIU,2024.10.12,UCNYi_zGmR519r5gYdOKLTjQ,Official music video for “Mantra” by JENNIE ou...,Music,PT2M28S,hd,20535235.0,152933.0,NaN,2021-01-13T06:19:55.86689Z,4.646152e+08,False,JENNIE
1,5ObJt_71AYc,2024.10.12,UCzU8-lZlRfkV3nj0RzAZdrQ,#Chile #Brasil #eliminatoriassudamericanas #Cr...,Sports,PT10M8S,hd,3966042.0,2549.0,Canal Oficial de la Confederación Sudamericana...,2014-02-19T20:24:31Z,3.990467e+08,False,CONMEBOL
2,zfb0whgBBA8,2024.10.12,UCgGYPnVJytkr6sVNLQ-l0zQ,"1,375,547 مشترك❤️\n\nـــــــــــــــــــــــــ...",Gaming,PT43M24S,hd,853167.0,10541.0,صانع محتوى وانزل مقاطع بشكل غير منظم 🌚,2012-08-01T16:24:26Z,1.143311e+08,False,LLE عبدالاله
3,SJfoPdeOPCQ,2024.10.12,UCzU8-lZlRfkV3nj0RzAZdrQ,#Venezuela #Argentina #eliminatoriassudamerica...,Sports,PT10M9S,hd,3758707.0,3115.0,Canal Oficial de la Confederación Sudamericana...,2014-02-19T20:24:31Z,3.990467e+08,False,CONMEBOL
4,UVb6QOKy0bI,2024.10.12,UCOzubmwpVZI7gD0Jf7Bk3Aw,Presenting The Official Trailer of Bougainvill...,Film & Animation,PT2M12S,hd,1730189.0,2869.0,Official YouTube Channel Of Amal Neerad Produc...,2017-05-31T14:31:01Z,1.999152e+07,False,Amal Neerad Productions
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1960532,QLoGHYbXcSY,2025.02.23,UC9-3c4LzdzT_HvW3Xuti9wg,Use discount code ‘CALFREEZY’ fo 5% off all pu...,Entertainment,PT30M12S,hd,1230188.0,1073.0,I make videos of me and my friends messing aro...,2010-04-19T21:10:29Z,8.815712e+08,False,Calfreezy
1960533,NkPMv21mN6o,2025.02.23,UCI1XS_GkLGDOgf8YLaaXNRA,Extrovert meeting an Introvert for the first t...,Comedy,PT2M36S,hd,802049.0,3468.0,NaN,2011-03-07T05:02:09Z,1.268135e+09,False,CalebCity
1960534,wjIcLkQCTvc,2025.02.23,UCTv-XvfzLX3i4IGWAm4sbmA,El CA Osasuna y el Real Madrid empatan a uno e...,Sports,PT3M34S,hd,3459667.0,3314.0,⚽ LA FUERZA DE NUESTRO FÚTBOL ⚽\n\n,2006-07-13T23:41:14Z,3.923950e+09,False,LALIGA EA SPORTS
1960535,0gPFoIUA1s4,2025.02.23,UC477Kvszl9JivqOxN1dFgPQ,I played Elden Ring Nightreign a couple weeks ...,Gaming,PT33M59S,hd,2227431.0,9882.0,I make Dark Souls videos. Some people think th...,2013-11-03T07:16:27Z,2.822633e+08,False,Iron Pineapple


In [14]:
#poularity basaed Recomendations

# **#poularity basaed Recomendations**

In [15]:
df1 = df[['video_id',"video_view_count","video_like_count","video_title","video_category_id"]]

In [16]:
df1.head()

,video_id,video_view_count,video_like_count,video_title,video_category_id
0,bB3-CUMERIU,20535235.0,2042255.0,JENNIE - Mantra (Official Music Video),Music
1,5ObJt_71AYc,3966042.0,2412822.5,CHILE vs. BRASIL [1-2] | RESUMEN | ELIMINATORI...,Sports
2,zfb0whgBBA8,853167.0,101155.0,تحدي الحقيبة مع ابو عبير 🔥 | سلسلتنا #2,Gaming
3,SJfoPdeOPCQ,3758707.0,2412822.5,VENEZUELA vs. ARGENTINA [1-1] | RESUMEN | ELIM...,Sports
4,UVb6QOKy0bI,1730189.0,67522.0,Bougainvillea Trailer | October 17 | Jyothirma...,Film & Animation


In [17]:
unique_count = len(df1['video_id'].unique())
print(unique_count)

59400


In [18]:
duplicate_values_count = df1.duplicated(subset=['video_id']).sum()
print(duplicate_values_count)

1901137


In [19]:
# Remove duplicates based on the 'video_id' column
df1 = df1.drop_duplicates(subset=['video_id'])

print(df1)

            video_id  video_view_count  video_like_count  \
0        bB3-CUMERIU        20535235.0         2042255.0   
1        5ObJt_71AYc         3966042.0         2412822.5   
2        zfb0whgBBA8          853167.0          101155.0   
3        SJfoPdeOPCQ         3758707.0         2412822.5   
4        UVb6QOKy0bI         1730189.0           67522.0   
...              ...               ...               ...   
1960032  78cdFYA9aow           86568.0            4328.0   
1960076  XBFX3pNQqAE          968457.0           18266.0   
1960138  ciOWuUDSTD4        12731217.0          187204.0   
1960141  7DWQ9ZfMpm8         4560304.0          103407.0   
1960146  Y_v4dx0UoXo        15385949.0          581917.0   

                                               video_title video_category_id  
0                   JENNIE - Mantra (Official Music Video)             Music  
1        CHILE vs. BRASIL [1-2] | RESUMEN | ELIMINATORI...            Sports  
2                  تحدي الحقيبة مع ابو عبي

In [20]:
avg_view = df1['video_view_count'].median()
avg_like = df1['video_like_count'].median()

In [21]:
avg_view

324051.0

In [22]:
popular_video = (df1["video_view_count"] > avg_view) & (df1['video_like_count'] > avg_like)

In [23]:
# Apply the filter to the DataFrame
popular_videos = df1[popular_video]

print(popular_videos)


            video_id  video_view_count  video_like_count  \
0        bB3-CUMERIU        20535235.0         2042255.0   
1        5ObJt_71AYc         3966042.0         2412822.5   
2        zfb0whgBBA8          853167.0          101155.0   
3        SJfoPdeOPCQ         3758707.0         2412822.5   
4        UVb6QOKy0bI         1730189.0           67522.0   
...              ...               ...               ...   
1959646  iCrHHfZoy3Q         1973470.0           23153.0   
1960076  XBFX3pNQqAE          968457.0           18266.0   
1960138  ciOWuUDSTD4        12731217.0          187204.0   
1960141  7DWQ9ZfMpm8         4560304.0          103407.0   
1960146  Y_v4dx0UoXo        15385949.0          581917.0   

                                               video_title video_category_id  
0                   JENNIE - Mantra (Official Music Video)             Music  
1        CHILE vs. BRASIL [1-2] | RESUMEN | ELIMINATORI...            Sports  
2                  تحدي الحقيبة مع ابو عبي

In [24]:
# Sort the popular videos by 'video_view_count' in descending order and get the top 50
popular_videos_sorted = popular_videos.sort_values('video_view_count', ascending=False).head(50)


In [25]:
popular_video

,0
0,True
1,True
2,True
3,True
4,True
...,...
1960032,False
1960076,True
1960138,True
1960141,True


In [26]:
df1.count()

,0
video_id,59399
video_view_count,59400
video_like_count,59400
video_title,59400
video_category_id,59400


In [27]:
df1[popular_video]

,video_id,video_view_count,video_like_count,video_title,video_category_id
0,bB3-CUMERIU,20535235.0,2042255.0,JENNIE - Mantra (Official Music Video),Music
1,5ObJt_71AYc,3966042.0,2412822.5,CHILE vs. BRASIL [1-2] | RESUMEN | ELIMINATORI...,Sports
2,zfb0whgBBA8,853167.0,101155.0,تحدي الحقيبة مع ابو عبير 🔥 | سلسلتنا #2,Gaming
3,SJfoPdeOPCQ,3758707.0,2412822.5,VENEZUELA vs. ARGENTINA [1-1] | RESUMEN | ELIM...,Sports
4,UVb6QOKy0bI,1730189.0,67522.0,Bougainvillea Trailer | October 17 | Jyothirma...,Film & Animation
...,...,...,...,...,...
1959646,iCrHHfZoy3Q,1973470.0,23153.0,1S vs BSS | FPL vs TDT | BOX vs FPT | VÒNG BẢN...,Gaming
1960076,XBFX3pNQqAE,968457.0,18266.0,Bir Gece Masalı 23. Bölüm 2. Fragmanı | “Raşit...,Entertainment
1960138,ciOWuUDSTD4,12731217.0,187204.0,这合照也太好玩了吧。#小丑 #天使 #超人不会飞 #shorts,People & Blogs
1960141,7DWQ9ZfMpm8,4560304.0,103407.0,Scary Teacher 3D Cutting Rainbow Candy Small P...,Gaming


In [28]:
# Sort the 'video_view_count' column in descending order
popular_video = df1[popular_video].sort_values('video_view_count',ascending=False)

In [29]:
popular_video.head(50)

,video_id,video_view_count,video_like_count,video_title,video_category_id
288123,UDIZhLy5HBY,307706319.0,4604401.0,Colorful Pasta Painting for Fun Times! 🍝 🎨,Howto & Style
5010,vMhfjd9E1Sc,170668174.0,3516285.0,How Strong is Tin Foil? 💪,Entertainment
1287669,wtyqjP8rHOE,168491888.0,2653836.0,Winter Bicycle Hack! Mom Saves the Day! ❄️🚲,Howto & Style
2487,KlyC6Du6VGY,142580283.0,2974815.0,GIANT Gummy Worm Pt.6 #shorts,Entertainment
1316403,G8845Wny3vI,136982508.0,1967618.0,Fun Game to Learn Emotions 😡😂😍,Entertainment
2453,nbzQdlWrcnk,134098365.0,4727636.0,Running With Bigger And Bigger Lunchlys,Entertainment
2218,k0QelMqdDpM,117925990.0,1617702.0,哈哈大家为了进去也是想尽办法！#火影忍者 #佐助 #家庭,People & Blogs
637616,dUVoJM3KxL4,117275578.0,1767844.0,Yay😃 Let's make a Cute Handbag for me 👜 #diycr...,Howto & Style
146,aRcUVhVlSHg,115563053.0,3539622.0,"Men Vs Women Survive The Wilderness For $500,000",Entertainment
2265,uNvimCdqdAk,112164458.0,1343115.0,Bike Vs Tricycle Fast Challenge,Entertainment


In [30]:
# Merge the DataFrames on 'video_id'
merged_df = popular_video.merge(df, on='video_id')

In [31]:
# Drop duplicates based on 'video_id'
merged_df = merged_df.drop_duplicates(subset=['video_id'])

In [32]:
merged_df.head(5)

,video_id,video_view_count_x,video_like_count_x,video_title_x,video_category_id_x,video_published_at,video_trending__date,video_trending_country,channel_id,video_title_y,...,channel_description,channel_custom_url,channel_published_at,channel_country,channel_view_count,channel_subscriber_count,channel_have_hidden_subscribers,channel_video_count,channel_localized_title,channel_localized_description
0,UDIZhLy5HBY,307706319.0,4604401.0,Colorful Pasta Painting for Fun Times! 🍝 🎨,Howto & Style,2024-10-12T10:00:29Z,2024.11.01,Azerbaijan,UCixexRrgWKp8lC-oW1s2RcQ,Colorful Pasta Painting for Fun Times! 🍝 🎨,...,FUNNY EDUCATIONAL SONGS THAT KIDS CAN LA LA LE...,@lalalearn9944,2021-04-15T12:59:34.129331Z,United States,1.916002e+09,2180000.0,False,314.0,La La Learn,FUNNY EDUCATIONAL SONGS THAT KIDS CAN LA LA LE...
470,vMhfjd9E1Sc,170668174.0,3516285.0,How Strong is Tin Foil? 💪,Entertainment,2024-09-06T20:00:22Z,2024.10.12,Malta,UC70Dib4MvFfT1tU6MqeyHpQ,How Strong is Tin Foil? 💪,...,"Hi, I’m Preston! I love me some Jesus and play...",@prestonyt,2012-07-04T07:01:56Z,United States,1.069656e+10,28200000.0,False,4162.0,Preston,"Hi, I’m Preston! I love me some Jesus and play..."
488,wtyqjP8rHOE,168491888.0,2653836.0,Winter Bicycle Hack! Mom Saves the Day! ❄️🚲,Howto & Style,2024-12-20T15:01:13Z,2025.01.08,Bangladesh,UCGmhKmPg9y420DQXdTA2a8A,Winter Bicycle Hack! Mom Saves the Day! ❄️🚲,...,KiddyLand Channel creates the most incredible ...,@kiddy-land,2023-02-21T10:20:16.849507Z,NaN,1.317635e+09,1700000.0,False,213.0,KiddyLand,KiddyLand Channel creates the most incredible ...
572,KlyC6Du6VGY,142580283.0,2974815.0,GIANT Gummy Worm Pt.6 #shorts,Entertainment,2024-09-16T08:01:50Z,2024.10.12,Bulgaria,UCeYTfGpNCmVhlxjQxBDrGPA,GIANT Gummy Worm Pt.6 #shorts,...,Comedic situations and and MUCH more!,@mrdegreeofficial,2017-01-31T13:26:49Z,United States,2.837128e+10,32700000.0,False,1771.0,Mr DegrEE,Comedic situations and and MUCH more!
751,G8845Wny3vI,136982508.0,1967618.0,Fun Game to Learn Emotions 😡😂😍,Entertainment,2024-12-27T15:00:30Z,2025.01.10,Azerbaijan,UCV63OnA8_yvaUEkD48N8zkQ,Fun Game to Learn Emotions 😡😂😍,...,Hi Moms and Dads!! Hi kids) \n🥳 Welcome to the...,@megandnancy,2024-01-26T01:47:37.904028Z,United States,1.531983e+09,1670000.0,False,246.0,Meg and Nancy - Learn and Play!,Hi Moms and Dads!! Hi kids) \n🥳 Welcome to the...


In [33]:
# Select specific columns
Popular_df = merged_df[["video_title_x", "video_category_id_x", "video_published_at", "video_trending__date", "video_trending_country", "channel_country", "channel_subscriber_count", "video_default_thumbnail"]]

print(Popular_df)

                                             video_title_x  \
0               Colorful Pasta Painting for Fun Times! 🍝 🎨   
470                              How Strong is Tin Foil? 💪   
488            Winter Bicycle Hack! Mom Saves the Day! ❄️🚲   
572                          GIANT Gummy Worm Pt.6 #shorts   
751                         Fun Game to Learn Emotions 😡😂😍   
...                                                    ...   
1362460                                 IST SIE SCHWANGER?   
1362489                        uważajcie na klucze #shorts   
1362504         Rio Da Yung OG - RIO FREE (Official Video)   
1362544                                     TÊNIS DE CAGAR   
1362557  Este Libro de Matemáticas tiene una PRECIOSA H...   

          video_category_id_x    video_published_at video_trending__date  \
0               Howto & Style  2024-10-12T10:00:29Z           2024.11.01   
470             Entertainment  2024-09-06T20:00:22Z           2024.10.12   
488             Howto & Sty

In [55]:
import pickle

# Assuming `recommended_videos` is already defined
with open('popular_df.pkl', 'wb') as f:
    pickle.dump(Popular_df, f)

# Collaborative Filtering Based Recommender System

In [34]:
df.head()

,video_id,video_published_at,video_trending__date,video_trending_country,channel_id,video_title,video_description,video_default_thumbnail,video_category_id,video_tags,...,channel_description,channel_custom_url,channel_published_at,channel_country,channel_view_count,channel_subscriber_count,channel_have_hidden_subscribers,channel_video_count,channel_localized_title,channel_localized_description
0,bB3-CUMERIU,2024-10-11T00:00:06Z,2024.10.12,United Arab Emirates,UCNYi_zGmR519r5gYdOKLTjQ,JENNIE - Mantra (Official Music Video),Official music video for “Mantra” by JENNIE ou...,https://i.ytimg.com/vi/bB3-CUMERIU/default.jpg,Music,"JENNIE,Mantra,제니,Jennie Mantra,Jennie from Bla...",...,NaN,@jennierubyjane,2021-01-13T06:19:55.86689Z,NaN,464615150.0,11600000.0,False,43.0,JENNIE,NaN
1,5ObJt_71AYc,2024-10-11T02:59:21Z,2024.10.12,United Arab Emirates,UCzU8-lZlRfkV3nj0RzAZdrQ,CHILE vs. BRASIL [1-2] | RESUMEN | ELIMINATORI...,#Chile #Brasil #eliminatoriassudamericanas #Cr...,https://i.ytimg.com/vi/5ObJt_71AYc/default.jpg,Sports,"fútbol,pelota,gol,copa,libertadores,sudamerica...",...,Canal Oficial de la Confederación Sudamericana...,@conmebol,2014-02-19T20:24:31Z,NaN,399046746.0,1610000.0,False,4637.0,CONMEBOL,Canal Oficial de la Confederación Sudamericana...
2,zfb0whgBBA8,2024-10-11T11:07:25Z,2024.10.12,United Arab Emirates,UCgGYPnVJytkr6sVNLQ-l0zQ,تحدي الحقيبة مع ابو عبير 🔥 | سلسلتنا #2,"1,375,547 مشترك❤️\n\nـــــــــــــــــــــــــ...",https://i.ytimg.com/vi/zfb0whgBBA8/default.jpg,Gaming,"تشكيله,فيفا,فيفا20,التمت,درافت,زلاتان,ام,جي,مل...",...,صانع محتوى وانزل مقاطع بشكل غير منظم 🌚,@mrlle99,2012-08-01T16:24:26Z,Saudi Arabia,114331110.0,1380000.0,False,314.0,LLE عبدالاله,صانع محتوى وانزل مقاطع بشكل غير منظم 🌚
3,SJfoPdeOPCQ,2024-10-11T00:10:10Z,2024.10.12,United Arab Emirates,UCzU8-lZlRfkV3nj0RzAZdrQ,VENEZUELA vs. ARGENTINA [1-1] | RESUMEN | ELIM...,#Venezuela #Argentina #eliminatoriassudamerica...,https://i.ytimg.com/vi/SJfoPdeOPCQ/default.jpg,Sports,"fútbol,pelota,gol,copa,libertadores,sudamerica...",...,Canal Oficial de la Confederación Sudamericana...,@conmebol,2014-02-19T20:24:31Z,NaN,399046746.0,1610000.0,False,4637.0,CONMEBOL,Canal Oficial de la Confederación Sudamericana...
4,UVb6QOKy0bI,2024-10-09T12:30:27Z,2024.10.12,United Arab Emirates,UCOzubmwpVZI7gD0Jf7Bk3Aw,Bougainvillea Trailer | October 17 | Jyothirma...,Presenting The Official Trailer of Bougainvill...,https://i.ytimg.com/vi/UVb6QOKy0bI/default.jpg,Film & Animation,NaN,...,Official YouTube Channel Of Amal Neerad Produc...,@amalneeradproductions5264,2017-05-31T14:31:01Z,NaN,19991522.0,40600.0,False,56.0,Amal Neerad Productions,Official YouTube Channel Of Amal Neerad Produc...


In [35]:
df2 = df[['video_id',"video_view_count","video_like_count","video_title","video_category_id","video_comment_count","channel_subscriber_count","video_category_id"]]

In [36]:
df2

,video_id,video_view_count,video_like_count,video_title,video_category_id,video_comment_count,channel_subscriber_count,video_category_id
0,bB3-CUMERIU,20535235.0,2042255.0,JENNIE - Mantra (Official Music Video),Music,152933.0,11600000.0,Music
1,5ObJt_71AYc,3966042.0,2412822.5,CHILE vs. BRASIL [1-2] | RESUMEN | ELIMINATORI...,Sports,2549.0,1610000.0,Sports
2,zfb0whgBBA8,853167.0,101155.0,تحدي الحقيبة مع ابو عبير 🔥 | سلسلتنا #2,Gaming,10541.0,1380000.0,Gaming
3,SJfoPdeOPCQ,3758707.0,2412822.5,VENEZUELA vs. ARGENTINA [1-1] | RESUMEN | ELIM...,Sports,3115.0,1610000.0,Sports
4,UVb6QOKy0bI,1730189.0,67522.0,Bougainvillea Trailer | October 17 | Jyothirma...,Film & Animation,2869.0,40600.0,Film & Animation
...,...,...,...,...,...,...,...,...
1960532,QLoGHYbXcSY,1230188.0,36921.0,I Tested Celebrity Owned Restaurants (USA Edit...,Entertainment,1073.0,4330000.0,Entertainment
1960533,NkPMv21mN6o,802049.0,117607.0,Trying to END a convo with someone who wants t...,Comedy,3468.0,4850000.0,Comedy
1960534,wjIcLkQCTvc,3459667.0,40966.0,CA OSASUNA 1 - 1 REAL MADRID I RESUMEN LALIGA ...,Sports,3314.0,12200000.0,Sports
1960535,0gPFoIUA1s4,2227431.0,78372.0,I played Elden Ring Nightreign... it's VERY weird,Gaming,9882.0,1630000.0,Gaming


In [37]:
unique_count = len(df2['video_id'].unique())
print(unique_count)

59400


In [38]:
duplicate_values_count = df2.duplicated(subset=['video_id']).sum()
print(duplicate_values_count)

1901137


In [39]:
# Remove duplicates based on the 'video_id' column
df2 = df2.drop_duplicates(subset=['video_id'])

print(df2)

            video_id  video_view_count  video_like_count  \
0        bB3-CUMERIU        20535235.0         2042255.0   
1        5ObJt_71AYc         3966042.0         2412822.5   
2        zfb0whgBBA8          853167.0          101155.0   
3        SJfoPdeOPCQ         3758707.0         2412822.5   
4        UVb6QOKy0bI         1730189.0           67522.0   
...              ...               ...               ...   
1960032  78cdFYA9aow           86568.0            4328.0   
1960076  XBFX3pNQqAE          968457.0           18266.0   
1960138  ciOWuUDSTD4        12731217.0          187204.0   
1960141  7DWQ9ZfMpm8         4560304.0          103407.0   
1960146  Y_v4dx0UoXo        15385949.0          581917.0   

                                               video_title video_category_id  \
0                   JENNIE - Mantra (Official Music Video)             Music   
1        CHILE vs. BRASIL [1-2] | RESUMEN | ELIMINATORI...            Sports   
2                  تحدي الحقيبة مع ابو 

In [40]:
avg_like = df2['video_like_count'].median()
avg_view = df2['video_view_count'].median()
avg_count = df2['video_comment_count'].median()


In [41]:
popular_video_2 = (df2["video_view_count"] > avg_view) & \
                (df2['video_like_count'] > avg_like) & \
                (df2['video_comment_count'] > avg_count)

In [42]:
df2[popular_video_2]

,video_id,video_view_count,video_like_count,video_title,video_category_id,video_comment_count,channel_subscriber_count,video_category_id
0,bB3-CUMERIU,20535235.0,2042255.0,JENNIE - Mantra (Official Music Video),Music,152933.0,11600000.0,Music
1,5ObJt_71AYc,3966042.0,2412822.5,CHILE vs. BRASIL [1-2] | RESUMEN | ELIMINATORI...,Sports,2549.0,1610000.0,Sports
2,zfb0whgBBA8,853167.0,101155.0,تحدي الحقيبة مع ابو عبير 🔥 | سلسلتنا #2,Gaming,10541.0,1380000.0,Gaming
3,SJfoPdeOPCQ,3758707.0,2412822.5,VENEZUELA vs. ARGENTINA [1-1] | RESUMEN | ELIM...,Sports,3115.0,1610000.0,Sports
4,UVb6QOKy0bI,1730189.0,67522.0,Bougainvillea Trailer | October 17 | Jyothirma...,Film & Animation,2869.0,40600.0,Film & Animation
...,...,...,...,...,...,...,...,...
1959388,Fu3NU4I70IA,690284.0,21877.0,Çocuklar yoldan geçen arabalara kar topu atı...,News & Politics,533.0,1000000.0,News & Politics
1959460,VaIkzdX5qJ8,643026.0,60547.0,Республіканець ВИСКАЗАВ усе ТРАМПУ щодо Путіна!,News & Politics,849.0,969000.0,News & Politics
1959541,u9shxHbYCs4,1406697.0,161583.0,The Return of Ducky Bhai 🔥,People & Blogs,18780.0,8260000.0,People & Blogs
1960076,XBFX3pNQqAE,968457.0,18266.0,Bir Gece Masalı 23. Bölüm 2. Fragmanı | “Raşit...,Entertainment,766.0,1080000.0,Entertainment


In [43]:
# Sort the 'video_view_count' column in descending order
popular_video_2 = df2[popular_video_2].sort_values('video_view_count',ascending=False)

In [44]:
popular_video_2

,video_id,video_view_count,video_like_count,video_title,video_category_id,video_comment_count,channel_subscriber_count,video_category_id
5010,vMhfjd9E1Sc,170668174.0,3516285.0,How Strong is Tin Foil? 💪,Entertainment,3858.0,28200000.0,Entertainment
2487,KlyC6Du6VGY,142580283.0,2974815.0,GIANT Gummy Worm Pt.6 #shorts,Entertainment,2677.0,32700000.0,Entertainment
2453,nbzQdlWrcnk,134098365.0,4727636.0,Running With Bigger And Bigger Lunchlys,Entertainment,7225.0,320000000.0,Entertainment
2218,k0QelMqdDpM,117925990.0,1617702.0,哈哈大家为了进去也是想尽办法！#火影忍者 #佐助 #家庭,People & Blogs,1753.0,3440000.0,People & Blogs
146,aRcUVhVlSHg,115563053.0,3539622.0,"Men Vs Women Survive The Wilderness For $500,000",Entertainment,49016.0,320000000.0,Entertainment
...,...,...,...,...,...,...,...,...
4876,iQ7HAy1G6Ag,324269.0,15971.0,Wrong Number - Khaligraph Jones x Shekinah Kar...,Music,1085.0,843000.0,Music
1451508,BPUFjunmNtU,324198.0,19187.0,Yapi - DONDE TE ESCONDES? (Video Oficial),Music,691.0,10300.0,Music
1250862,egr8ngkNfAg,324177.0,24835.0,On a survécu en Mer comme dans RAFT... (avec u...,Entertainment,1935.0,3000000.0,Entertainment
1718239,YFsX_-_wB5s,324167.0,36797.0,IST SIE SCHWANGER?,Entertainment,617.0,2800000.0,Entertainment


In [45]:
pivot_table = popular_video_2.pivot_table(
    index='video_id',
    columns='video_category_id',
    values='video_comment_count',
    aggfunc='sum'  # or 'mean', 'count', etc.
)

ValueError: Grouper for 'video_category_id' not 1-dimensional

In [ ]:
popular_video_2.pivot_table(index='video_id',columns='video_category_id',values='video_comment_count')

In [46]:
# Selecting relevant numerical features for similarity computation
features = ['video_view_count', 'video_like_count', 'video_comment_count']
video_features = popular_video_2[['video_title'] + features].fillna(0)
video_features.set_index('video_title', inplace=True)

In [47]:
 #Construct feature matrix and compute similarity
matrix = csr_matrix(video_features.values)
similarity_matrix = cosine_similarity(matrix)

In [49]:
popular_video_2

,video_id,video_view_count,video_like_count,video_title,video_category_id,video_comment_count,channel_subscriber_count,video_category_id
5010,vMhfjd9E1Sc,170668174.0,3516285.0,How Strong is Tin Foil? 💪,Entertainment,3858.0,28200000.0,Entertainment
2487,KlyC6Du6VGY,142580283.0,2974815.0,GIANT Gummy Worm Pt.6 #shorts,Entertainment,2677.0,32700000.0,Entertainment
2453,nbzQdlWrcnk,134098365.0,4727636.0,Running With Bigger And Bigger Lunchlys,Entertainment,7225.0,320000000.0,Entertainment
2218,k0QelMqdDpM,117925990.0,1617702.0,哈哈大家为了进去也是想尽办法！#火影忍者 #佐助 #家庭,People & Blogs,1753.0,3440000.0,People & Blogs
146,aRcUVhVlSHg,115563053.0,3539622.0,"Men Vs Women Survive The Wilderness For $500,000",Entertainment,49016.0,320000000.0,Entertainment
...,...,...,...,...,...,...,...,...
4876,iQ7HAy1G6Ag,324269.0,15971.0,Wrong Number - Khaligraph Jones x Shekinah Kar...,Music,1085.0,843000.0,Music
1451508,BPUFjunmNtU,324198.0,19187.0,Yapi - DONDE TE ESCONDES? (Video Oficial),Music,691.0,10300.0,Music
1250862,egr8ngkNfAg,324177.0,24835.0,On a survécu en Mer comme dans RAFT... (avec u...,Entertainment,1935.0,3000000.0,Entertainment
1718239,YFsX_-_wB5s,324167.0,36797.0,IST SIE SCHWANGER?,Entertainment,617.0,2800000.0,Entertainment


In [50]:
# Get recommendations for a specific video
video_id = "Rio Da Yung OG - RIO FREE (Official Video)"  # Example video ID
if video_id not in video_features.index:
    raise ValueError("Video ID not found in dataset.")


In [51]:
video_index = video_features.index.get_loc(video_id)
similarity_scores = list(enumerate(similarity_matrix[video_index]))
sorted_videos = sorted(similarity_scores, key=lambda x: x[1], reverse=True)[1:6]
recommended_videos = [video_features.index[i] for i, _ in sorted_videos]

In [52]:
print(recommended_videos)

['GOT7 (갓세븐)의 킬링보이스를 라이브로! – A, 딱 좋아, 니가 하면, 하드캐리, PYTHON, 하지하지마, Lullaby, 고마워 | 딩고뮤직 | Dingo Music', 'SEVENTEEN(세븐틴) - 동그라미(Encircled)', 'JENNIE & Dominic Fike - Love Hangover (Official Video)', 'jschlatt — Santa Claus Is Coming To Town (Official)', 'Matuê - Isso é Sério ft. Brandão']


In [54]:
import pickle

# Assuming `recommended_videos` is already defined
with open('recommended_videos.pkl', 'wb') as f:
    pickle.dump(recommended_videos, f)